In [1]:
1

1

In [1]:
import torch
import math
from typing_extensions import Annotated, List
import torch.nn as nn 
class PositionalEncoding(nn.Module):
    def __init__(self, dmodel:Annotated[int, "Dimensions of the embeddings"]) ->None:
        super().__init__()
        self.dmodel = dmodel


    def forward(self, pos:Annotated[int, "Position of the word"]) -> torch.Tensor:
        position_encoding_vector = torch.zeros(self.dmodel)
        for i in range(0, self.dmodel, 2):
            div_term = math.pow(10000, i / self.dmodel)
            position_encoding_vector[i] = math.sin(pos / div_term)
            if i + 1 < self.dmodel:
                position_encoding_vector[i + 1] = math.cos(pos / div_term)
            
        return position_encoding_vector.unsqueeze(0)

            

            





In [2]:
pe = PositionalEncoding(6)
pe.forward(1)

tensor([[0.8415, 0.5403, 0.0464, 0.9989, 0.0022, 1.0000]])

In [3]:
torch.arange(0, 5, dtype=torch.float).unsqueeze(1)

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.]])

In [4]:
torch.exp(torch.arange(0, 4, 2).float() * (-math.log(10000.0) / 4))

tensor([1.0000, 0.0100])

In [7]:
torch.exp(torch.arange(0, 4, 2).float() * math.log(10000) / 4)

tensor([  1.0000, 100.0000])

In [11]:
torch.arange(36).view(2, 3, 6)

tensor([[[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]],

        [[18, 19, 20, 21, 22, 23],
         [24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35]]])

In [14]:
torch.arange(0, 10).unsqueeze(1).float() 

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.],
        [9.]])

In [21]:
torch.arange(54).view(2, 3, 9).transpose(0, 1) #(3, 2, 9)

tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
         [27, 28, 29, 30, 31, 32, 33, 34, 35]],

        [[ 9, 10, 11, 12, 13, 14, 15, 16, 17],
         [36, 37, 38, 39, 40, 41, 42, 43, 44]],

        [[18, 19, 20, 21, 22, 23, 24, 25, 26],
         [45, 46, 47, 48, 49, 50, 51, 52, 53]]])

In [2]:
import torch
import math
import torch.nn as nn
from typing_extensions import Annotated


class PositionalEncoding(nn.Module):
    def __init__(self,
                 max_len: Annotated[int, "It means how many no of words are there in a sequence"],
                 d_model: Annotated[int, "It tells in how many dimension each and every word represents"]) -> None:
        super().__init__()
        positional_encoder = torch.zeros(
            max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(
            1).float()  # (max_len, 1)

        div_term = torch.exp(torch.arange(0, d_model, 2).float(
        ) * (-math.log(10000.0) / d_model))  # (d_model//2,)

        positional_encoder[:, 0::2] = torch.sin(position * div_term)
        positional_encoder[:, 1::2] = torch.cos(position * div_term)

        # Shape it to (1, max_len, d_model) for broadcasting with input: (batch_size, seq_len, d_model)
        positional_encoder = positional_encoder.unsqueeze(0)

        self.register_buffer("positional_encoder", positional_encoder)

    def forward(self, input_data: torch.Tensor) -> torch.Tensor:
        """
        input_data: shape (batch_size, seq_len, d_model)
        returns: same shape with positional encoding added
        """
        seq_len = input_data.size(1)
        return input_data + self.positional_encoder[:, :seq_len]


### Synthetic data (rows = 100, seq_len = 10, embed_dim = 8)

In [8]:
rows = 100
seq_len = 10
embed_dim = 8
batch = 2
x = torch.arange(rows * seq_len * embed_dim).reshape(rows, seq_len, embed_dim)
batch_data = x[:batch]
batch_data.shape

torch.Size([2, 10, 8])

In [9]:
pe = PositionalEncoding(seq_len, embed_dim)

In [10]:
pe(batch_data)

tensor([[[  0.0000,   2.0000,   2.0000,   4.0000,   4.0000,   6.0000,   6.0000,
            8.0000],
         [  8.8415,   9.5403,  10.0998,  11.9950,  12.0100,  14.0000,  14.0010,
           16.0000],
         [ 16.9093,  16.5839,  18.1987,  19.9801,  20.0200,  21.9998,  22.0020,
           24.0000],
         [ 24.1411,  24.0100,  26.2955,  27.9553,  28.0300,  29.9995,  30.0030,
           32.0000],
         [ 31.2432,  32.3464,  34.3894,  35.9211,  36.0400,  37.9992,  38.0040,
           40.0000],
         [ 39.0411,  41.2837,  42.4794,  43.8776,  44.0500,  45.9987,  46.0050,
           48.0000],
         [ 47.7206,  49.9602,  50.5646,  51.8253,  52.0600,  53.9982,  54.0060,
           56.0000],
         [ 56.6570,  57.7539,  58.6442,  59.7648,  60.0699,  61.9976,  62.0070,
           64.0000],
         [ 64.9894,  64.8545,  66.7174,  67.6967,  68.0799,  69.9968,  70.0080,
           72.0000],
         [ 72.4121,  72.0889,  74.7833,  75.6216,  76.0899,  77.9960,  78.0090,
           